In [2]:
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import NamedStyle, Font, PatternFill, Alignment
import os

# Die folgenden imports stammen aus der Datei ../00_common/common.py

import sys
sys.path.append("../00_common")
import common

def check_fingerprint(wb, matrnr):
    ws = wb["1) ABC-Analyse"]
    if ws["U12"].value == int(matrnr):
        return "OK"
    else:
        return "NOK"

def check_exercise_1_1(wb):
    # Check formatting of cells A1:K1  (only check B1 and J1)
    ws = wb["1) ABC-Analyse"]
    points = 0
    # Check if background color is FH-Mint
    if ws["B1"].fill.start_color.index == ws["J1"].fill.start_color.index == "FF00B1AC":
        points += 1

    # Check, if the numberformat in cells B3 and C10 has one number after the decimal point
    if ws["B3"].number_format == r'_-* #,##0.00\ "€"_-;\-* #,##0.00\ "€"_-;_-* "-"??\ "€"_-;_-@_-': # Excel dt "Buchhaltung"
        points += 1
    if ws["B3"].number_format == r'#,##0.00\ "€"':                                                  # Excel dt "Währung"
        points += 1
    if ws["B3"].number_format == r'0.00 "€"':
        points += 1
    if ws["C10"].number_format == r'_-* #,##0.00\ "€"_-;\-* #,##0.00\ "€"_-;_-* "-"??\ "€"_-;_-@_-': # Excel dt "Buchhaltung"
        points += 1
    if ws["C10"].number_format == r'#,##0.00\ "€"':                                                  # Excel dt "Währung"
        points += 1
    if ws["C10"].number_format == r'0.00 "€"':
        points += 1  

    # Check if number-format in column A is "Kund-"0000
    if ws["A10"].number_format == r'"Kund-"0000':
        points += 1 
        
    return points

def check_exercise_1_2(wb):
    # Check, if turnover data is sorted by turnover
    ws = wb["1) ABC-Analyse"]
    l_turnover = []
    for row in range(2, ws.max_row + 1):
        cellname = "B"+str(row)
        l_turnover.append(ws[cellname].value)
    control_list = sorted(l_turnover, reverse = True)
    if l_turnover == control_list:
        return 2
    else: 
        return 0

def check_exercise_1_3(wb, wb_val):
    # Check, if cumulated column contains the sum of all original values in the last cell and the max value in the first cell
    ws = wb["1) ABC-Analyse"]
    ws_val = wb_val["1) ABC-Analyse"]
    l_turnover = []
    for row in range(2, ws.max_row + 1):
        cellname = "B"+str(row)
        l_turnover.append(ws[cellname].value)
    cellname = "C" + str(ws.max_row)
    if ws_val["C2"].value != None and ws_val[cellname].value != None:
        if round(ws_val["C2"].value, 2) == round(max(l_turnover), 2) and round(ws_val[cellname].value, 2) == round(sum(l_turnover), 2):
            return 2
    else:
        return 0

def check_exercise_1_4(wb):
    # Check, if formula for cumulative values is used with autofill, check, if there is a $
    ws = wb["1) ABC-Analyse"]
    if ws["C3"].value != None:
        formula = ws["C3"].value
        formula = formula.replace(" ", "")
        if formula == "=C2+B3" or formula == "=B3+C2" or formula == "=SUM(B$2:B3)" or formula == "=SUM($B$2:B3)":
            return 2
        else:
            print("unbekannte Formel", formula)
            return 0
    return 0

def check_exercise_1_5(wb):
    # Check if the class (A, B, C) is calculated using the VLOOKUP() function.
    ws = wb["1) ABC-Analyse"]
    formula = ws["E2"].value
    if formula != None:
        formula = formula.replace(" ", "")
        if formula[:8] == "=VLOOKUP" and formula.count("$") >= 1:
            return 2
        elif formula[:3] == "=IF" and formula.count("$") >= 1:
            return 1
        else:
            return 0
    return 0

def check_exercise_1_6(wb_val):
    # Check if the values for the calculation of the ABC-Analysis are correct
    ws = wb_val["1) ABC-Analyse"]
    l_turnover = []
    for row in range(2, ws.max_row + 1):
        cellname = "B"+str(row)
        l_turnover.append(ws[cellname].value)
    l_turnover = sorted(l_turnover, reverse = True)
    sum_turnover = sum(l_turnover)
    accumulated = 0
    l_abc = []
    AB_border = ws["G3"].value
    BC_border = ws["G4"].value
    for turnover in l_turnover:
        accumulated += turnover
        accumulated_perc = accumulated / sum_turnover
        if accumulated_perc <= AB_border:
            abc_class = "A"
        elif accumulated_perc <= BC_border:
            abc_class = "B"
        else:
            abc_class = "C"
        l_abc.append((turnover, accumulated, accumulated_perc, abc_class))
    abc_count = {"A" : 0, "B" : 0, "C": 0}
    abc_turnover = {"A" : 0, "B" : 0, "C": 0}
    for cust in l_abc:
        if cust[3] == "A":
            abc_count["A"] += 1
            abc_turnover["A"] += cust[0]
        elif cust[3] == "B":
            abc_count["B"] += 1
            abc_turnover["B"] += cust[0]
        elif cust[3] == "C":
            abc_count["C"] += 1
            abc_turnover["C"] += cust[0]

    abc_turnover["A"] = round(abc_turnover["A"], 2)
    abc_turnover["B"] = round(abc_turnover["B"], 2)
    abc_turnover["C"] = round(abc_turnover["C"], 2)

    points = 0

    if ws["J2"].value == abc_count["A"] and ws["J3"].value == abc_count["B"] and ws["J4"].value == abc_count["C"]:
        points += 2
    if ws["K2"].value != None and ws["K3"].value != None and ws["K4"].value != None:
        if round(ws["K2"].value, 2) == abc_turnover["A"] and round(ws["K3"].value, 2) == abc_turnover["B"] and round(ws["K4"].value, 2) == abc_turnover["C"]:
            points += 2
    
    return points

def check_exercise_2_1(wb):
    # Check formatting of cells A1:K1  (only check B1 and J1)
    ws = wb["2) Klausur"]
    points = 0
    # Check if background color is FH-Mint
    if ws["D2"].fill.start_color.index == ws["J4"].fill.start_color.index == ws["P2"].fill.start_color.index == "FF00B1AC":
        points += 1

    # Check, if font-size in row 1 is 16
    if ws["A1"].font.size == ws["N1"].font.size == 16:
        points += 1

    return points

def check_exercise_2_2(wb):
    # Check, if stud-ids (matr) are sorted
    ws = wb["2) Klausur"]
    l_matr = []
    for row in range(3, ws.max_row + 1):
        cellname = "A"+str(row)
        if ws[cellname].value != None:
            l_matr.append(ws[cellname].value)
    control_list = sorted(l_matr)
    if l_matr == control_list and len(l_matr) == ws.max_row - 2:
        return 2
    else: 
        return 0

def check_exercise_2_3(wb):
    # Check if the VLOOKUP Function is used correctly. Check Formula in Cell D6
    ws = wb["2) Klausur"]
    points = 0
    if ws["D6"].value != None:
        formula = ws["D6"].value 
        if "VLOOKUP" in formula:
            points += 1
        formula = formula.split(",")
        if formula[0][9:] == "A6":
            points += 1
        if formula[1] == "N:T" or formula[1].count("$") >= 2:
            points += 1
        if formula[2] == "7":
            points += 1
        if formula[3] == "FALSE)":
            points += 1
    return points
    
def check_exercise_2_4(wb):
    # Check if the VLOOKUP Function is used correctly. Check Formula in Cell H11
    ws = wb["2) Klausur"]
    points = 0
    if ws["H11"].value != None:
        formula = ws["H11"].value 
        if "VLOOKUP" in formula:
            points += 1
        formula = formula.split(",")
        if formula[0][9:] == "G11":
            points += 1
        if formula[1] == "J:L" or formula[1].count("$") >= 2:
            points += 1
        if formula[3] == "TRUE)":
            points += 1
    return points

def check_exercise_2_5(wb):
    # Check if the Hilfstabelle, column J is designed correctly
    ws = wb["2) Klausur"]
    points = 0
    if ws["J5"].value == 0:
        points += 1
    if ws["J6"].value in ["=K2", "=$K$2", "=$K2", "=K$2"]:
        points += 1
    if ws["J7"].value in ["=J6+$K$3", "=J6+K$3"]:
        points += 1
    return points

def check_exercise_2_6(wb):
    # Check if the Hilfstabelle uses COUNTIF() in column L
    ws = wb["2) Klausur"]
    points = 0
    if ws["L8"].value != None:
        if "COUNTIF" in ws["L8"].value:
            points += 1
        if "K8" in ws["L8"].value:
            points += 1
    return points


         
def check_workbook(matrnr, wb, wb_val):
    points = [matrnr]
    points.append(check_fingerprint(wb, matrnr))
    points.append(check_exercise_1_1(wb))
    points.append(check_exercise_1_2(wb))
    points.append(check_exercise_1_3(wb, wb_val))
    points.append(check_exercise_1_4(wb))
    points.append(check_exercise_1_5(wb))
    points.append(check_exercise_1_6(wb_val))
    points.append(check_exercise_2_1(wb))
    points.append(check_exercise_2_2(wb))
    points.append(check_exercise_2_3(wb))
    points.append(check_exercise_2_4(wb))
    points.append(check_exercise_2_5(wb))
    points.append(check_exercise_2_6(wb))

    print(points)
    
    return points



l_points = common.check_ExcelFiles_in_Abgaben(check_workbook)
header = ["MatrNr", "Fingerprint", "1 Format", "1 Sortiert", "1 Kumluiert", "1 Autofill", "1 SVERWEIS", "1 ABC-Tabelle", "2 Format", "2 Sortiert", "2 SVW1", "2 SVW2", "2 Hilfst", "2 Countif"] 
common.result_into_file(l_points, header)


[3035252, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2957432, 'OK', 3, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0]
[3999610, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3907393, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[4037627, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2054917, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3756363, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1459993, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[4176899, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2457378, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3748162, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[4322701, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3196143, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2369135, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[4110346, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3660895, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2097324, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2209422, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1574969, 'OK', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
os.chdir(".")
print(os.getcwd())